In [3]:
import os
from PIL import Image
import time
import json
import yaml

In [66]:
# Image Resize (width: 640): 너무 작다.

def resize_image_to_width_640(filename):
    with Image.open(filename) as img:
        width, height = img.size
        ratio = height / width
        
        new_width = 640
        new_height = int(ratio * new_width)

        # Image.Dither.None 옵션을 사용하여 속도 약 2배 빨라짐
        im_resize = img.resize((new_width, new_height), Image.Dither.NONE)

    os.remove(filename)
    im_resize.save(filename)

In [69]:
# Image Resize (height: 640)

def resize_image_to_height_640(filename):
    with Image.open(filename) as img:
        width, height = img.size
        ratio = width / height
        
        new_height = 640
        new_width = int(ratio * new_height)

        # Image.Dither.None 옵션을 사용하여 속도 약 2배 빨라짐
        im_resize = img.resize((new_width, new_height), Image.Dither.NONE)

    os.remove(filename)
    im_resize.save(filename)

In [70]:
# Images Resize by recursive

def resize_images_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            resize_images_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.jpg':
                    resize_image_to_height_640(os.path.join(src, filename))

In [72]:
# Resize Train Data

start = time.time()
resize_images_recursive('./data/train/원천데이터')
print("time :", time.time() - start)

time : 7316.926134109497


In [71]:
# Resize Validation Data

start = time.time()
resize_images_recursive('./data/validation/원천데이터')
print("time :", time.time() - start)

time : 910.5802710056305


In [4]:
# Get Classes List

model_classes = []

def get_model_classes(src):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
        
    rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:18]
    if rawDataID not in model_classes:
        model_classes.append(rawDataID)

In [5]:
# Get Classes List by recursive

def get_classes_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            get_classes_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.json':
                    get_model_classes(os.path.join(src, filename))

In [6]:
# Get Classes

start = time.time()
get_classes_recursive('./data/train/labels')
print("time :", time.time() - start)

time : 31.181153059005737


In [7]:
model_classes[0]

'KI_056_17'

In [8]:
sorted_model_classes = sorted(model_classes)
sorted_model_classes[0]

'HY_025_17'

In [9]:
len(sorted_model_classes)

142

In [1]:
all_parts = { "P00":"차량전체", "P01":"프론트범퍼", "P02":"리어범퍼", "P03":"타이어(휠)", "P04":"A필러",
       "P05":"C필러", "P06":"사이드미러", "P07":"앞도어", "P08":"뒷도어", "P09":"라디에이터그릴",
       "P10":"헤드램프", "P11":"리어램프", "P12":"보닛", "P13":"트렁크", "P14":"루프", }

In [2]:
ignore_parts = ["P03", "P04", "P05", "P06", "P07", "P08", "P12", "P13", "P14", ]

In [21]:
# Make Label with parts

def make_label_with_parts(src):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
    
    rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:18]
    resolution = jess_dict['rawDataInfo']['resolution']
    width, height = int(resolution.split('*')[0]), int(resolution.split('*')[1])
    
    used_class_id = 0
    with open(os.path.splitext(src)[0]+'.txt', 'w') as f:
        for obj in jess_dict['learningDataInfo']['objects']:
            classID = obj['classId'].split('.')[0]
            if classID not in ignore_parts:
                used_class_id += 1
                cx = (int(obj['coords']['tl']['x']) + int(obj['coords']['tr']['x'])) / 2 / width
                cy = (int(obj['coords']['tl']['y']) + int(obj['coords']['bl']['y'])) / 2 / height
                w = (int(obj['coords']['tr']['x']) - int(obj['coords']['tl']['x'])) / width
                h = (int(obj['coords']['bl']['y']) - int(obj['coords']['tl']['y'])) / height

                txt = str(sorted_model_classes.index(rawDataID)) + ' ' + str(cx) + ' ' + str(cy) + ' ' + str(w) + ' ' + str(h) + '\n'

                f.write(txt)    
                
    if used_class_id == 0:
        os.remove(os.path.splitext(src)[0]+'.txt')
        img_path = os.path.splitext(src)[0].replace("labels", "images")
        os.remove(img_path+'.jpg')
    
    os.remove(src)

In [22]:
# Make Labels with parts by recursive

def make_labels_with_parts_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            make_labels_with_parts_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.json':
                    make_label_with_parts(os.path.join(src, filename))

In [24]:
# Make Labels for train

start = time.time()
make_labels_with_parts_recursive('./data/train/labels')
print("time :", time.time() - start)

time : 190.24099564552307


In [23]:
# Make Labels for validation

start = time.time()
make_labels_with_parts_recursive('./data/validation/labels')
print("time :", time.time() - start)

time : 24.15755796432495


In [25]:
# Make dict from sorted_model_classes

dict_classes = {i : c for i, c in enumerate(sorted_model_classes)}

In [26]:
# Make yaml file for YOLOv5

yaml_file = 'car_model_ignore_parts.yaml'

yaml_data = dict(
    path = "YOLO_DATA_IGNORE_PARTS/data",
    train = "train",
    val = "validation",
    nc = len(dict_classes),
    names = dict_classes
)

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f, explicit_start = True, default_flow_style = False)

In [27]:
dict_classes

{0: 'HY_025_17',
 1: 'HY_025_18',
 2: 'HY_025_19',
 3: 'HY_041_17',
 4: 'HY_041_18',
 5: 'HY_041_19',
 6: 'HY_041_20',
 7: 'HY_041_21',
 8: 'HY_042_20',
 9: 'HY_050_17',
 10: 'HY_050_18',
 11: 'HY_053_19',
 12: 'HY_053_20',
 13: 'HY_053_21',
 14: 'HY_054_17',
 15: 'HY_054_18',
 16: 'HY_054_19',
 17: 'HY_054_20',
 18: 'HY_054_21',
 19: 'HY_058_17',
 20: 'HY_058_18',
 21: 'HY_058_19',
 22: 'HY_058_20',
 23: 'HY_058_21',
 24: 'HY_063_17',
 25: 'HY_063_18',
 26: 'HY_063_19',
 27: 'HY_063_20',
 28: 'HY_063_21',
 29: 'HY_064_17',
 30: 'HY_064_18',
 31: 'HY_064_19',
 32: 'HY_064_20',
 33: 'HY_064_21',
 34: 'HY_067_17',
 35: 'HY_067_18',
 36: 'HY_067_19',
 37: 'HY_067_20',
 38: 'HY_067_21',
 39: 'HY_069_17',
 40: 'HY_069_18',
 41: 'HY_069_19',
 42: 'HY_069_20',
 43: 'HY_069_21',
 44: 'HY_073_17',
 45: 'HY_073_18',
 46: 'HY_073_19',
 47: 'HY_083_17',
 48: 'HY_083_18',
 49: 'HY_083_19',
 50: 'HY_083_20',
 51: 'HY_083_21',
 52: 'HY_091_17',
 53: 'HY_091_18',
 54: 'HY_091_19',
 55: 'HY_091_20',
 5